# Utilisation de jit

<img src="figures/numba_blue_icon_rgb.png" alt="Drawing" style="width: 20%;"/>

<center>**Loic Gouarin**</center>
<center>*6 juillet 2017*</center>

Comme dit en introduction, Numba propose plusieurs fonctionnalités. La plus générale est la fonction jit. Nous allons dans la suite aborder l'ensemble des possibilités de celle-ci.

Numba est efficace sur des données numériques de type scalaire ou des tableaux NumPy. Contrairement à vos habitudes pour une utilisation efficace de NumPy (écriture vectorielle), il est indispensable de dérouler les boucles comme vous le feriez dans un langage bas niveau pour avoir de bonnes performances. 

Prenons l'exemple d'un calcul sommant les coefficients d'un vecteur. En NumPy, ça s'écrit

In [1]:
import numpy as np

x = np.random.rand(1000000)
print(x.sum())

500089.407052


Le temps de calcul est

In [2]:
%timeit x.sum()

1000 loops, best of 3: 1.18 ms per loop


Voyons ce que ça donne avec Numba.

In [3]:
from numba import jit

In [4]:
@jit
def numba_sum(x):
    res= 0
    for i in range(x.size):
        res += x[i]
    return res

In [5]:
%timeit numba_sum(x)

The slowest run took 62.96 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 5.05 ms per loop


Le message indiqué en première ligne est dû au fait que Numba compile la fonction lors de son premier appel. Du coup, l'exécution de la fonction *numba_sum* est beaucoup plus lente que les autres.

Les performances sont bien plus mauvaises avec Numba. Ceci est dû au fait que NumPy utilise déjà des fonctions optimisées pour faire la somme.

## Fonctions utiles

Le décorateur jit de Numba ajoute un certain nombre de fonctions offrant des informations pouvant s'avérer très utiles.

In [6]:
@jit
def somme(a, b):
    return a + b

#### inspect_types

Cette méthode permet de savoir quelles sont les fonctions générées pour un certain type de données d'entrée tout en nous montrant l'inférence de type proposée par Numba.

In [7]:
somme.inspect_types()

Nous voyons ici qu'il n'y a pas pour le moment de fonction compilée par Numba. Si nous l'appelons une première fois avec un certain type de données

In [8]:
somme(1, 2)
somme.inspect_types()

somme (int64, int64)
--------------------------------------------------------------------------------
# File: <ipython-input-6-d0b9178a02d8>
# --- LINE 1 --- 
# label 0
#   del b
#   del a
#   del $0.3

@jit

# --- LINE 2 --- 

def somme(a, b):

    # --- LINE 3 --- 
    #   a = arg(0, name=a)  :: int64
    #   b = arg(1, name=b)  :: int64
    #   $0.3 = a + b  :: int64
    #   $0.4 = cast(value=$0.3)  :: int64
    #   return $0.4

    return a + b




In [9]:
somme(1., 2.)
somme.inspect_types()

somme (int64, int64)
--------------------------------------------------------------------------------
# File: <ipython-input-6-d0b9178a02d8>
# --- LINE 1 --- 
# label 0
#   del b
#   del a
#   del $0.3

@jit

# --- LINE 2 --- 

def somme(a, b):

    # --- LINE 3 --- 
    #   a = arg(0, name=a)  :: int64
    #   b = arg(1, name=b)  :: int64
    #   $0.3 = a + b  :: int64
    #   $0.4 = cast(value=$0.3)  :: int64
    #   return $0.4

    return a + b


somme (float64, float64)
--------------------------------------------------------------------------------
# File: <ipython-input-6-d0b9178a02d8>
# --- LINE 1 --- 
# label 0
#   del b
#   del a
#   del $0.3

@jit

# --- LINE 2 --- 

def somme(a, b):

    # --- LINE 3 --- 
    #   a = arg(0, name=a)  :: float64
    #   b = arg(1, name=b)  :: float64
    #   $0.3 = a + b  :: float64
    #   $0.4 = cast(value=$0.3)  :: float64
    #   return $0.4

    return a + b




#### inspect_llvm

Vous pouvez également avoir accès à la représentation intermédiaire de LLVM.

In [10]:
for k, v in somme.inspect_llvm().items():
    print(v)

; ModuleID = 'somme'
source_filename = "<string>"
target datalayout = "e-m:e-i64:64-f80:128-n8:16:32:64-S128"
target triple = "x86_64-unknown-linux-gnu"

@.const.somme = internal constant [6 x i8] c"somme\00"
@".const.Fatal error: missing _dynfunc.Closure" = internal constant [38 x i8] c"Fatal error: missing _dynfunc.Closure\00"
@PyExc_RuntimeError = external global i8
@".const.missing Environment" = internal constant [20 x i8] c"missing Environment\00"

; Function Attrs: norecurse nounwind
define i32 @"_ZN8__main__9somme$242Exx"(i64* noalias nocapture %retptr, { i8*, i32 }** noalias nocapture readnone %excinfo, i8* noalias nocapture readnone %env, i64 %arg.a, i64 %arg.b) local_unnamed_addr #0 {
entry:
  %.15 = add nsw i64 %arg.b, %arg.a
  store i64 %.15, i64* %retptr, align 8
  ret i32 0
}

define i8* @"_ZN7cpython8__main__9somme$242Exx"(i8* %py_closure, i8* %py_args, i8* nocapture readnone %py_kws) local_unnamed_addr {
entry:
  %.5 = alloca i8*, align 8
  %.6 = alloca i8*, align 8
  

#### inspect_asm

Vous pouvez également avoir accès à l'assembleur qui intervient dans la phase finale du processus.

In [11]:
for k, v in somme.inspect_asm().items():
    print(v)

	.text
	.file	"somme"
	.globl	_ZN8__main__9somme$242Exx
	.p2align	4, 0x90
	.type	_ZN8__main__9somme$242Exx,@function
_ZN8__main__9somme$242Exx:
	addq	%r8, %rcx
	movq	%rcx, (%rdi)
	xorl	%eax, %eax
	retq
.Lfunc_end0:
	.size	_ZN8__main__9somme$242Exx, .Lfunc_end0-_ZN8__main__9somme$242Exx

	.globl	_ZN7cpython8__main__9somme$242Exx
	.p2align	4, 0x90
	.type	_ZN7cpython8__main__9somme$242Exx,@function
_ZN7cpython8__main__9somme$242Exx:
	.cfi_startproc
	pushq	%r15
.Lcfi0:
	.cfi_def_cfa_offset 16
	pushq	%r14
.Lcfi1:
	.cfi_def_cfa_offset 24
	pushq	%r12
.Lcfi2:
	.cfi_def_cfa_offset 32
	pushq	%rbx
.Lcfi3:
	.cfi_def_cfa_offset 40
	subq	$24, %rsp
.Lcfi4:
	.cfi_def_cfa_offset 64
.Lcfi5:
	.cfi_offset %rbx, -40
.Lcfi6:
	.cfi_offset %r12, -32
.Lcfi7:
	.cfi_offset %r14, -24
.Lcfi8:
	.cfi_offset %r15, -16
	movq	%rdi, %rbx
	movabsq	$.const.somme, %r10
	movabsq	$PyArg_UnpackTuple, %r11
	leaq	16(%rsp), %r8
	leaq	8(%rsp), %r9
	movl	$2, %edx
	movl	$2, %ecx
	xorl	%eax, %eax
	movq	%rsi, %rdi
	movq	%r10, %rsi
	c

## Rappeler sa fonction Python initiale

Il est toujours possibe d'appeler la fonction Python initiale sans la couche Numba.

In [12]:
somme.py_func(1, 2)

3

## Spécialiser sa fonction pour des types données

Si vous utilisez juste **@jit**, Numba créera pour vous une nouvelle fonction dès lors qu'il n'a pas à disposition la fonction compilée avec ces types. Vous pouvez néanmoins avoir envie que votre fonction ne marche que pour certains types et vous renvoie une erreur si les types ne sont pas supportés.

Prenons par exemple le produit, nous voulons que celui-ci ne fonctionne que sur des entiers scalaires ou des tableaux 1d d'entiers. 

In [13]:
from numba import jit

@jit(['int32[:](int32[:], int32[:])',
      'int32(int32, int32)'])
def produit(a, b):
    return a*b

In [14]:
produit(2, 3)

6

In [15]:
produit(3., 2.2)

6

In [16]:
import numpy as np

a = np.arange(10, dtype=np.int32)
b = np.arange(10, dtype=np.int32)

produit(a, b)

array([ 0,  1,  4,  9, 16, 25, 36, 49, 64, 81], dtype=int32)

In [17]:
import numpy as np

a = np.random.random(10)
b = np.random.random(10)

produit(a, b)

TypeError: No matching definition for argument type(s) array(float64, 1d, C), array(float64, 1d, C)

Les types reconnus par Numba sont les suivants

- void,
- intp, uintp,
- intc, uintc,
- int8, uint8, int16, uint16, int32, uint32, int64, uint64,
- float32, float64,
- complex64, complex128.

Pour les tableaux, il suffit de préciser chacune des dimensions par le caractère deux points (**:**). Par exemple

- float32[:] est un tableau de type float du langage C à une dimension,
- float64[:, :] est un tableau de type double du langage C à deux dimensions.

## Options de compilation

Numba permet de mettre certaines directives suplémentaires dans les arguments du décorateur permettant de changer le comportement de la compilation. Pour le décorateur **@jit**, il en existe trois

- **nopython** 

cette option indique qu'il n'y a pas d'objets Python dans la fonction à optimiser et que tout peut donc être fait en bas niveau. Si c'est le cas, le résultat est optimal. Essayez toujours de mettre ce mode. La compilation échouera si la fonction comporte des objets Python.

- **nogil** 

cette option permet de relâcher le Global Interpreter Lock (GIL). Elle est utilisée lorsque l'on veut utiliser la fonction en parallèle en utilisant des threads.

- **cache** 

si vous ne voulez pas que Numba recompile votre fonction à chaque lancement de votre script, vous pouvez utiliser cette fonctionnalité qui met dans un fichier la version optimisée.

## inlining

Il est possible d'appeler des fonctions optimisées par **@jit** à l'intérieur de fonctions faisant également appel à **jit**. Si c'est possible, Numba les remplace alors directement par les lignes de codes associées.

Par exemple

In [18]:
import math
from numba import njit

@njit
def square(x):
    return x ** 2

@njit
def hypot(x, y):
    return math.sqrt(square(x) + square(y))

In [19]:
hypot(2., 3.)

3.605551275463989

In [20]:
for k, v in hypot.inspect_asm().items():
    print(v)

	.text
	.file	"hypot"
	.globl	_ZN8__main__9hypot$248Edd
	.p2align	4, 0x90
	.type	_ZN8__main__9hypot$248Edd,@function
_ZN8__main__9hypot$248Edd:
	vmulsd	%xmm0, %xmm0, %xmm0
	vmulsd	%xmm1, %xmm1, %xmm1
	vaddsd	%xmm1, %xmm0, %xmm0
	vsqrtsd	%xmm0, %xmm0, %xmm0
	vmovsd	%xmm0, (%rdi)
	xorl	%eax, %eax
	retq
.Lfunc_end0:
	.size	_ZN8__main__9hypot$248Edd, .Lfunc_end0-_ZN8__main__9hypot$248Edd

	.globl	_ZN7cpython8__main__9hypot$248Edd
	.p2align	4, 0x90
	.type	_ZN7cpython8__main__9hypot$248Edd,@function
_ZN7cpython8__main__9hypot$248Edd:
	.cfi_startproc
	pushq	%r15
.Lcfi0:
	.cfi_def_cfa_offset 16
	pushq	%r14
.Lcfi1:
	.cfi_def_cfa_offset 24
	pushq	%r13
.Lcfi2:
	.cfi_def_cfa_offset 32
	pushq	%r12
.Lcfi3:
	.cfi_def_cfa_offset 40
	pushq	%rbx
.Lcfi4:
	.cfi_def_cfa_offset 48
	subq	$32, %rsp
.Lcfi5:
	.cfi_def_cfa_offset 80
.Lcfi6:
	.cfi_offset %rbx, -48
.Lcfi7:
	.cfi_offset %r12, -40
.Lcfi8:
	.cfi_offset %r13, -32
.Lcfi9:
	.cfi_offset %r14, -24
.Lcfi10:
	.cfi_offset %r15, -16
	movq	%rdi, %rbx
	movabsq	

## Exercice

Dans toute la suite de ce tutoriel, nous allons optimiser un ensemble de fonctions travaillant sur des splines cubiques. Il ne sera pas nécessaire de connaître les maths qu'il y a derrière et nous ne rentrerons donc pas dans les détails (si ça vous intéresse, vous pouvez suivre ce [lien](http://www.aip.de/groups/soe/local/numres/bookcpdf/c3-3.pdf)).

Le choix de cette exercice vient du site [inconvergent](http://inconvergent.net/generative). En prenant une figure géométrique de départ et en y associant une spline cubique passant par un ensemble de points discrétisant celle-ci, il est possible par petites perturbations successives d'avoir de très jolies images. Comme ces deux exemples.

![Sand Spline](figures/sandspline.png)

Le code source est dicponible [ici](https://github.com/gouarin/cours_numba_2017/blob/master/examples/sandspline/origin.py).

Lorsque l'on utilise des splines cubiques, il est nécessaire de calculer la dérivée seconde passant par un ensemble de points. La fonction qui permet de les calculer est la suivante.

In [ ]:
def cubic_spline(x, y):
    n = x.shape[0]
    u = np.zeros_like(y)
    y2 = np.zeros_like(y)

    dif = np.diff(x)
    sig = dif[:-1]/(x[2:]-x[:-2])
    
    u[1:-1] = (y[2:]- y[1:-1])/dif[1:] - (y[1:-1]-y[:-2])/dif[:-1]

    for i in range(1, n-1):
        p = sig[i-1]*y2[i-1] + 2.
        y2[i] = (sig[i-1]-1)/p
        u[i] = (6*u[i]/(x[i+1]-x[i-1])-sig[i-1]*u[i-1])/p
    
    for i in range(n-2, -1, -1):
        y2[i] = y2[i]*y2[i+1]+u[i]

    return y2

#### Exercice 1

Essayer d'optimiser cette fonction en mettant juste **@jit**.

#### Exercice 2

Essayer de mettre **@jit(nopython=True)** ou **@njit**. Qu'en déduisez-vous ?

#### Exercice 3

Essayer d'améliorer les performances en aidant un peu plus Numba à optimiser cette fonction. Quel est le gain ?

In [21]:
# execute this part to modify the css style
from IPython.core.display import HTML
def css_styling():
    styles = open("./style/custom.css").read()
    return HTML(styles)
css_styling()